In [32]:
import yfinance as yf
inda = yf.download("INDA", start="2021-01-01", end="2025-3-31")


[*********************100%***********************]  1 of 1 completed


In [33]:
inda.columns = [col[0] if isinstance(col, tuple) else col for col in inda.columns]

In [34]:
inda

,Close,High,Low,Open,Volume
Date,,,,,
2021-01-04,37.514973,38.064063,37.514973,38.017527,4324800
2021-01-05,38.194355,38.194355,37.905849,37.933767,3571800
2021-01-06,38.073364,38.250189,37.803472,37.803472,4333500
2021-01-07,37.896545,38.017529,37.850009,37.887237,1914500
2021-01-08,38.789974,38.789974,38.417708,38.613150,3823900
...,...,...,...,...,...
2025-03-24,52.119999,52.150002,52.020000,52.070000,5524500
2025-03-25,52.040001,52.070000,51.709999,51.919998,10997300
2025-03-26,51.400002,51.660000,51.400002,51.599998,8790500


In [35]:
import pandas_ta as ta

In [36]:
inda.ta.strategy("All",append=False, timed=True)

63it [00:03, 18.48it/s]


AttributeError: 'Series' object has no attribute 'append'

In [26]:
inda

,Close,High,Low,Open,Volume,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,ACCBL_20,...,KSTs_9,KURT_30,KVO_34_55_13,KVOs_34_55_13,LR_14,LOGRET_1,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,MAD_30
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,37.514973,38.064063,37.514973,38.017527,4324800,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,38.194355,38.194355,37.905849,37.933767,3571800,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.017948,NaN,NaN,NaN,NaN
2021-01-06,38.073364,38.250189,37.803472,37.803472,4333500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.003173,NaN,NaN,NaN,NaN
2021-01-07,37.896545,38.017529,37.850009,37.887237,1914500,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.004655,NaN,NaN,NaN,NaN
2021-01-08,38.789974,38.789974,38.417708,38.613150,3823900,38.085155,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.023302,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-24,52.119999,52.150002,52.020000,52.070000,5524500,50.929333,51.453536,50.405131,0.524202,48.343597,...,-2644.667171,0.449784,898416.402724,336270.779400,50.915164,0.012550,0.307323,0.456776,-0.149453,0.795445
2025-03-25,52.040001,52.070000,51.709999,51.919998,10997300,51.342667,51.859256,50.826078,0.516589,48.472636,...,-2136.936821,0.445090,607233.824953,374979.785908,51.399253,-0.001536,0.434270,0.466978,-0.032708,0.850600
2025-03-26,51.400002,51.660000,51.400002,51.599998,8790500,51.530667,52.055483,51.005851,0.524816,48.589643,...,-1577.110432,0.025953,388544.697970,376917.630488,51.662967,-0.012374,0.477726,0.408348,0.069379,0.898200
